In [1]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.layers import MaxPooling2D

2023-11-05 15:30:27.796803: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
img_rows, img_cols = 224, 224

def create_model():
    inputs = Input((img_rows, img_cols, 3))
    x = Conv2D(96, (11, 11), padding="same", activation="relu")(inputs)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Dropout(0.3)(x)

    x = Conv2D(256, (5, 5), padding="same", activation="relu")(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Dropout(0.3)(x)

    x = Conv2D(384, (3, 3), padding="same", activation="relu")(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Dropout(0.3)(x)

    pooledOutput = GlobalAveragePooling2D()(x)
    pooledOutput = Dense(1024)(pooledOutput)
    outputs = Dense(128)(pooledOutput)

    model = Model(inputs, outputs)
    return model

In [3]:
from tensorflow.keras.layers import Lambda
from tensorflow.keras import backend as K

feature_extractor = create_model()
imgA = Input(shape=(img_rows, img_cols, 3))
imgB = Input(shape=(img_rows, img_cols, 3))
featA = feature_extractor(imgA)
featB = feature_extractor(imgB)

def euclidean_distance(vectors):
    (featA, featB) = vectors
    sum_squared = K.sum(K.square(featA - featB), axis=1, keepdims=True)
    return K.sqrt(K.maximum(sum_squared, K.epsilon()))

distance = Lambda(euclidean_distance)([featA, featB])
outputs = Dense(1, activation="sigmoid")(distance)
model = Model(inputs=[imgA, imgB], outputs=outputs)

In [4]:
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [5]:
import numpy as np

def generate_train_image_pairs(images_dataset, labels_dataset):
    unique_labels = np.unique(labels_dataset)
    label_wise_indices = dict()
    for label in unique_labels:
        label_wise_indices.setdefault(label,
                                      [index for index, curr_label in enumerate(labels_dataset) if
                                       label == curr_label])
    
    pair_images = []
    pair_labels = []
    for index, image in enumerate(images_dataset):
        pos_indices = label_wise_indices.get(labels_dataset[index])
        pos_image = images_dataset[np.random.choice(pos_indices)]
        pair_images.append((image, pos_image))
        pair_labels.append(1)

        neg_indices = np.where(labels_dataset != labels_dataset[index])
        neg_image = images_dataset[np.random.choice(neg_indices[0])]
        pair_images.append((image, neg_image))
        pair_labels.append(0)
    return np.array(pair_images), np.array(pair_labels)

In [6]:
import glob
import cv2
from tensorflow.keras.utils import img_to_array

# pull images from david/ folder
images_dataset = []
labels_dataset = []
images_test_dataset = []
labels_test_dataset = []

for image_path in glob.glob("train/*.jpg"):
    image = cv2.imread(image_path)
    #print(image.shape)
    # need to bounding box here
    image = cv2.resize(image, (img_rows, img_cols))
    image = img_to_array(image)
    images_dataset.append(image)
    label = image_path.split("/")[-1].split("_")[0]
    labels_dataset.append(label)

for image_path in glob.glob("test/*.jpg"):
    image = cv2.imread(image_path)
    print(image.shape, image_path)
    # need to bounding box here
    image = cv2.resize(image, (img_rows, img_cols))
    image = img_to_array(image)
    images_test_dataset.append(image)
    label = "david" if "test_david" in image_path else "other"
    labels_test_dataset.append(label)

(4032, 3024, 3) test/test_david10.jpg
(600, 434, 3) test/test_will_smith.jpg


In [7]:
## Model training

images_pair, labels_pair = generate_train_image_pairs(images_dataset, labels_dataset)
history = model.fit([images_pair[:, 0], images_pair[:, 1]], labels_pair[:],validation_split=0.1,batch_size=4,epochs=5)

Epoch 1/5
7/7 [==============================] - 57s 7s/step - loss: 5.0137 - accuracy: 0.5000 - val_loss: 0.8351 - val_accuracy: 0.2500
Epoch 2/5
7/7 [==============================] - 43s 6s/step - loss: 1.2663 - accuracy: 0.5000 - val_loss: 0.7405 - val_accuracy: 0.2500
Epoch 3/5
7/7 [==============================] - 40s 6s/step - loss: 0.7408 - accuracy: 0.5000 - val_loss: 0.7171 - val_accuracy: 0.2500
Epoch 4/5
7/7 [==============================] - 39s 6s/step - loss: 0.6988 - accuracy: 0.5714 - val_loss: 0.6942 - val_accuracy: 0.5000
Epoch 5/5
7/7 [==============================] - 36s 5s/step - loss: 0.6936 - accuracy: 0.5000 - val_loss: 0.6943 - val_accuracy: 0.5000


# TESTING

In [8]:
def generate_test_image_pairs(images_dataset, labels_dataset, image):
    unique_labels = np.unique(labels_dataset)
    label_wise_indices = dict()
    for label in unique_labels:
        label_wise_indices.setdefault(label, [index for index, curr_label in enumerate(labels_dataset) if label == curr_label])
  
    pair_images = []
    pair_labels = []
    for label, indices_for_label in label_wise_indices.items():
        test_image = images_dataset[np.random.choice(indices_for_label)]
        pair_images.append((image, test_image))
        pair_labels.append(label)
    return np.array(pair_images), np.array(pair_labels)

In [9]:
def predict(image):
  test_image_pairs, test_label_pairs = generate_test_image_pairs(images_dataset, labels_dataset, image) # produce an array of test image pairs and test label pairs
  #print("Test pairs", test_image_pairs)
  print("Test labels", test_label_pairs)
  for index, pair in enumerate(test_image_pairs):
      pair_image1 = np.expand_dims(pair[0], axis=-1)
      pair_image1 = np.expand_dims(pair_image1, axis=0)
      pair_image2 = np.expand_dims(pair[1], axis=-1)
      pair_image2 = np.expand_dims(pair_image2, axis=0)
      prediction = model.predict([pair_image1, pair_image2])[0][0]
      print("Test image pair {} similarity: {:.2f}".format(index, prediction))

In [10]:
print(labels_test_dataset)

# show the first image
predict(images_test_dataset[0])

predict(images_test_dataset[1])


['david', 'other']
Test labels ['david' 'will-smith']


1/1 [==============================] - 1s 1s/step
Test image pair 0 similarity: 0.50
1/1 [==============================] - 0s 366ms/step
Test image pair 1 similarity: 0.50
Test labels ['david' 'will-smith']
1/1 [==============================] - 0s 367ms/step
Test image pair 0 similarity: 0.50
1/1 [==============================] - 0s 441ms/step
Test image pair 1 similarity: 0.50
